In [2]:
# Install the transformers library
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.1 MB/s eta 0:00:00


In [3]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [4]:
# Check if the GPU can be detected
import tensorflow as tf

device_name = tf.test.gpu_device_name()              # '/device:GPU:0' means GPU is enabled
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
import torch
# torch.cuda.is_available = lambda : False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [6]:
# Load tokenizer and model, create trainer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [7]:
# Create list of texts (can be imported from .csv, .xls etc.)
pred_texts = ['I like that','That is annoying','This is great!','Wouldn´t recommend it.']

In [8]:
# Example: Import data from csv-file stored on Google Drive

#from google.colab import drive
#drive.mount('/content/drive')


#file_name = "/content/drive/MyDrive/Colab Notebooks/your-filename.csv"
#text_column = "text"

#df_pred = pd.read_csv(file_name)
#pred_texts = df_pred[text_column].dropna().astype('str').tolist()

In [9]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [10]:
# Run predictions
predictions = trainer.predict(pred_dataset)

***** Running Prediction *****
  Num examples = 4
  Batch size = 8


In [11]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [12]:
# Create DataFrame with texts, predictions, labels, and scores
df = pd.DataFrame(list(zip(pred_texts,preds,labels,scores)), columns=['text','pred','label','score'])
df.head()

,text,pred,label,score
0,I like that,1,POSITIVE,0.998657
1,That is annoying,0,NEGATIVE,0.999409
2,This is great!,1,POSITIVE,0.998727
3,Wouldn´t recommend it.,0,NEGATIVE,0.999486


In [13]:
df = pd.read_csv('/S22_total.csv', index_col=0).drop_duplicates()
df.head()

,Title,Content,Class,Flair,Date,Thirdparty,Display,Battery,Camera,Noti,Connect,Messages
0,Should i purchase a S21Ultra with the S22 arou...,Alright guys boomer here. My whole life i was ...,N,None,2022-01-04 05:34:00,False,False,False,False,False,True,False
1,S22 Ultra In-Hands Delivery Date? USA,I'm curious if there's any information about w...,N,None,2022-01-04 05:34:00,False,False,False,False,False,False,False
2,S22 Ultra has morphed in the Note Series!.,Has Samsung basically rebranded the S22 Ultra ...,N,None,2022-01-04 05:34:00,False,False,False,False,False,False,False
3,Who is planning to trade in their S21 for a S22?,"Last year, Samsung put out amazing pre-order d...",N,None,2022-01-04 05:34:00,False,False,False,False,False,False,False
4,So... who's getting an S22?,I know it's unlikely to make any difference in...,N,None,2022-01-04 05:34:00,False,False,True,False,False,False,False


In [14]:
# filter training data to desired dates (after 2022.2.22)
from datetime import datetime, timedelta
df = df[df.Date.apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S').date() > datetime(2022,2,21).date())]

# format training data to Title, Content and Class
df = df.loc[: , 'Title':'Class']
df.head()

,Title,Content,Class
452,Zfold 3 or S22 ultra,Trying to decide between the zfold 3 or s22 ul...,N
453,S22 video cam,Anyone tried out the video camera on the s22 y...,N
454,Thinking about trading my S21Ultra for S22+,Every year the hype about the camera on these ...,N
455,S21 Ultra vs S22+,Both phones are currently at the same price fo...,N
456,S21 or S22 Base/Standard Model,"Hey All,\n\nLooking to pick up either the S21 ...",N


In [15]:
# Define Title + Content concatenator
from timeit import default_timer as timer
import re

def data_concat(df22, R_known=True):

    # Split the input dataframe into Text (Title + Contents) and Classes dataframes
    # input must have 3 columns of string entries (Title, Contents, and Classes)

    # check if every row entry of each column is string type (some are NaN, so no)
    # print(df22.applymap(lambda x : type(x).__name__).eq({'Title': 'str', 'Content': 'str', 'Class':'str'}))

    # convert NaN to empty strings (NaN -> str)
        # df22.apply(str) converts all columns to str, as well
    df22 = df22.replace(float('nan'), '', regex=True)

    # concatenate strings of title & content with a " " in between (1 body of text)
    df22['Text'] = df22['Title'] + " " + df22['Content']      # slicing DataFrame via .iloc[:,0] makes it a Series
    df22 = df22.loc[: , ['Text', 'Class']]    # so initialize it as a DataFrame. pd.DataFrame(some_Series) works

    if R_known == True:
    # R, r, YR = 1;     N, n, YN = 0
        R_cases = re.compile('R|YR', re.IGNORECASE)
        N_cases = re.compile('N|YN', re.IGNORECASE)
        df22['Class'] = df22['Class'].replace(to_replace=R_cases, value=1)
        df22['Class'] = df22['Class'].replace(to_replace=N_cases, value=0)
    else:
        # R_known == False; prepping not yet classified data
        Y_N_cases = re.compile('Y|N', re.IGNORECASE)
        df22['Class'] = df22['Class'].replace(to_replace=Y_N_cases, value=0)     # all N's for simplicity

    df22['Class'] = df22['Class'].astype('int32')


    return df22


In [16]:
df = data_concat(df)     # run only once!
df.head()

,Text,Class
452,Zfold 3 or S22 ultra Trying to decide between ...,0
453,S22 video cam Anyone tried out the video camer...,0
454,Thinking about trading my S21Ultra for S22+ Ev...,0
455,S21 Ultra vs S22+ Both phones are currently at...,0
456,"S21 or S22 Base/Standard Model Hey All,\n\nLoo...",0


In [17]:
df['Class'].value_counts()

0    17752
1     5218
Name: Class, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments

model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--siebert--sentiment-roberta-large-english/snapshots/6eac71655a474ee4d6d0eee7fa532300c537856d/config.json
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type

In [19]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
         

In [20]:
# model = model.to( torch.device('cuda') )     # need NVIDIA driver for 'cuda'; currently have AMD on work laptop
# model = model.to('cpu')         # train on CPU

model = model.to('cuda')          # or  model.cuda()


In [21]:

test_data = ["This is possibly the worst battery I have ever seen on a mobile device",
            "How is my device running so smoothly?"]
tokenizer(test_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[0, 713, 16, 3544, 5, 2373, 3822, 38, 33, 655, 450, 15, 10, 1830, 2187, 2], [0, 6179, 16, 127, 2187, 878, 98, 17359, 116, 2, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [22]:
X = list(df["Text"])
y = list(df["Class"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [23]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'attention_mask'])

In [24]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [25]:
len(X_train),len(X_val)


(18376, 4594)

In [26]:
# Create torch dataset
class VOC_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [27]:
train_dataset = VOC_Dataset(X_train_tokenized, y_train)
val_dataset = VOC_Dataset(X_val_tokenized, y_val)

In [28]:
train_dataset[5]


{'input_ids': tensor([    0, 41513, 29465,  7462, 11468,  1669,     6, 50141,  1437, 50140,
         50118,   100,   524,  2114,    41,   696,     6,   147,    77,    38,
           860,     7,   185,    10,  2170,    11,   614,  1109,  1274,    19,
          7462,    11,     5, 11477,  1553,     5,  1553, 35084,     8,     5,
          7462, 10117,    15,   734, 10385,  1302,     7,  1369,   454,    38,
          1370,  5001,     5,  1553,     4,  6142,    64,   244,   116,  1437,
         50140, 50118, 13987,    47,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [29]:
def compute_metrics(m):
    print(type(m))
    pred, labels = m
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
     

In [30]:
# Define Trainer
args = TrainingArguments(
    # report_to = 'wandb',                     # enable logging to W&B
    output_dir="output",                     # output directory
    num_train_epochs=1,
    per_device_train_batch_size=1,
    # overwrite_output_dir = True,
    # evaluation_strategy = 'steps',          # check evaluation metrics at each epoch
    # learning_rate = 5e-5,                   # we can customize learning rate
    # max_steps = 30000,
    # logging_steps = 100,                    # we will log every 100 steps
    # eval_steps = 5000,                      # we will perform evaluation every 500 steps
    # save_steps = 10000,
    # load_best_model_at_end = True,
    # metric_for_best_model = 'accuracy',
    # run_name = 'custom_training'            # name of the W&B run

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,            # for padding batched data
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
# clear cache before training
torch.cuda.empty_cache()


In [32]:
# https://github.com/facebookresearch/fairseq/issues/1933
# https://huggingface.co/spaces/OFA-Sys/OFA-Generic_Interface/blob/8fc1d8aafce5821301443744696303bac6227f52/fairseq/examples/roberta/commonsense_qa/README.md

# PyTorch fairseq

MAX_UPDATES=9000      # Number of training steps, where   training step = training set / batch size. Originally 3000
WARMUP_UPDATES=150    # Linearly increase LR over this many steps
MAX_EPOCH=1           # Number of training epochs.
LR=3e-05              # Peak LR for fixed LR scheduler. 1e-05 default
NUM_CLASSES=2
MAX_SENTENCES=1       # Batch size per GPU.
UPDATE_FREQ=1         # Accumulate gradients to simulate training on 8 GPUs. 
DATA_DIR='VOC_final_output'
ROBERTA_PATH='/SiERoBERT_large/model.pt'

! CUDA_VISIBLE_DEVICES=0 fairseq-train $DATA_DIR --ddp-backend=no_c10d \
  --restore-file $ROBERTA_PATH \
  --reset-optimizer --reset-dataloader --reset-meters \
  --no-epoch-checkpoints --no-last-checkpoints --no-save-optimizer-state \
  #--best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
  --task sentence_ranking \
  --num-classes $NUM_CLASSES \
  --init-token 0 --separator-token 2 \
  --max-option-length 128 \
  --max-positions 512 \
  --truncate-sequence \
  --arch roberta_large \
  --dropout 0.1 --attention-dropout 0.1 --weight-decay 0.01 \
  --criterion sentence_ranking \
  --optimizer adam --adam-betas '(0.9, 0.98)' --adam-eps 1e-06 \
  --clip-norm 0.0 \
  --lr-scheduler fixed --lr $LR \
  --warmup-updates $WARMUP_UPDATES --total-num-update $MAX_UPDATES \
  --memory-efficient-fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 32 \
  --batch-size $MAX_SENTENCES \
  --required-batch-size-multiple 1 \
  --update-freq $UPDATE_FREQ \
  --max-epoch $MAX_EPOCH
  --log-interval 1000 \

# --memory-efficient-fp16 instead of --fp16 solves 'CUDA out of memory' problem, but slow training
# --max-sentences $MAX_SENTENCES , not batch-size
# --fp16-scale-window 128 default

/bin/bash: fairseq-train: command not found


In [35]:
torch.cuda.memory_summary(device=None, abbreviated=False)


'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    5422 MB |    7106 MB |  149798 MB |  144376 MB |\n|       from large pool |    5417 MB |    7077 MB |  146794 MB |  141377 MB |\n|       from small pool |       4 MB |      30 MB |    3004 MB |    2999 MB |\n|---------------------------------------------------------------------------|\n| Active memory         |    5422 MB |    7106 MB |  149798 MB |  144376 MB |\n|       from large pool |    5417 MB |    7077 MB |

In [34]:
from timeit import default_timer as timer
start = timer()

trainer.train()


end = timer()
print("%4f seconds, %4f minutes elapsed" % (float(end-start), float((end-start)/60)))

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18376
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 18376
  Number of trainable parameters = 355361794


Step,Training Loss


KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()


In [ ]:
np.set_printoptions(suppress=True)


In [ ]:
text = "Super charging is working very well."
# text = "so many issues with this phone."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)    # also gelu(), silu()
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

In [ ]:
trainer.save_model('SiebertPractice')

In [ ]:
# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

In [ ]:
model_2 = AutoModelForSequenceClassification.from_pretrained('SiebertPractice')
model_2.to('cuda')

In [ ]:
# text = "Super charging is working very well."
text = "so many issues with this phone."
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

In [ ]:

#today's VOC
# import file and combine title & content
voctoday = pd.read_csv('/20230124.csv', index_col=0).drop_duplicates()
voctoday = data_concat(voctoday.loc[: , 'Title':'Class'])

voclist = list(voctoday['Text'])
predlist = []

for i in range(len(voclist)):
  inputs = tokenizer(voclist[i], padding = True, truncation = True, return_tensors='pt').to('cuda')
  outputs = model_2(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  predictions = predictions.cpu().detach().numpy()
  predlist.append(predictions)

organizedzip = zip([x[:25] for x in voclist],
                   predlist,
                   voctoday['class'])
    
# results as voc text, predictions, and actual value
print(list(organizedzip))
